In [1]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
SHOW = True

In [2]:
df = pd.read_csv('US_Accidents_Dec20_updated.csv')
df

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2716600,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,40.11206,-83.03187,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2716601,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,39.86501,-84.04873,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-2716602,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,39.10209,-84.52396,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-2716603,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,39.09841,-84.52241,0.219,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
4,A-2716604,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,41.06217,-81.53547,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.00248,-117.37936,33.99888,-117.37094,0.543,At Market St - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
1516060,A-4239403,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.76696,-117.14806,32.76555,-117.15363,0.338,At Camino Del Rio/Mission Center Rd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
1516061,A-4239404,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.77545,-117.84779,33.77740,-117.85727,0.561,At Glassell St/Grand Ave - Accident. in the ri...,...,False,False,False,False,False,False,Day,Day,Day,Day
1516062,A-4239405,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.99246,-118.40302,33.98311,-118.39565,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [10]:
df1 = df[['ID','Start_Time','County','State','Severity','Start_Lat','Start_Lng']]
df1['Start_Time']= pd.to_datetime(df['Start_Time'])
df1['Weekday']= df1['Start_Time'].dt.weekday # Monday = 0, Sunday = 6
df1['Hour'] = df1['Start_Time'].dt.hour
df1

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,ID,Start_Time,County,State,Severity,Start_Lat,Start_Lng,Weekday,Hour
0,A-2716600,2016-02-08 00:37:08,Franklin,OH,3,40.10891,-83.09286,0,0
1,A-2716601,2016-02-08 05:56:20,Montgomery,OH,2,39.86542,-84.06280,0,5
2,A-2716602,2016-02-08 06:15:39,Hamilton,OH,2,39.10266,-84.52468,0,6
3,A-2716603,2016-02-08 06:15:39,Hamilton,OH,2,39.10148,-84.52341,0,6
4,A-2716604,2016-02-08 06:51:45,Summit,OH,2,41.06213,-81.53784,0,6
...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2019-08-23 18:03:25,Riverside,CA,2,34.00248,-117.37936,4,18
1516060,A-4239403,2019-08-23 19:11:30,San Diego,CA,2,32.76696,-117.14806,4,19
1516061,A-4239404,2019-08-23 19:00:21,Orange,CA,2,33.77545,-117.84779,4,19
1516062,A-4239405,2019-08-23 19:00:21,Los Angeles,CA,2,33.99246,-118.40302,4,19


In [13]:
df1 = df1.dropna()

In [14]:
print(df1.dtypes)

ID                    object
Start_Time    datetime64[ns]
County                object
State                 object
Severity               int64
Start_Lat            float64
Start_Lng            float64
Weekday                int64
Hour                   int64
dtype: object


In [15]:
##################### DATA CLEANING #####################
df1.to_csv("data_final_project.csv")  

In [47]:

df2 = df1['State'].value_counts().rename_axis('State').to_frame('Counts')
df2

,Counts
State,
CA,448833
FL,153007
OR,87484
TX,75142
NY,60974
MN,52345
VA,51198
NC,50159
PA,42844


In [63]:
##################### Accidents count per State #####################
df2.to_csv('Accident_Count_Per_State.csv')

In [64]:
df_CA = df1[df1['State'] =='CA']
df_FL = df1[df1['State'] =='FL']
df_TX = df1[df1['State'] =='TX']
df_GA = df1[df1['State'] =='GA']

df_CA_count = df_CA['County'].value_counts().rename_axis('County').to_frame('Counts')
df_CA_count["State"] = 'CA'

df_FL_count = df_FL['County'].value_counts().rename_axis('County').to_frame('Counts')
df_FL_count["State"] = 'FL'

df_TX_count = df_TX['County'].value_counts().rename_axis('County').to_frame('Counts')
df_TX_count["State"] = 'TX'

df_GA_count = df_GA['County'].value_counts().rename_axis('County').to_frame('Counts')
df_GA_count["State"] = 'GA'

In [65]:
result = pd.concat([df_CA_count,df_FL_count,df_TX_count,df_GA_count], axis=0)

In [66]:
result

,Counts,State
County,,
Los Angeles,138819,CA
San Bernardino,30251,CA
Orange,29670,CA
San Diego,26623,CA
Sacramento,25941,CA
...,...,...
Schley,2,GA
Glascock,1,GA
Wilcox,1,GA


In [67]:
result.to_csv('Accident_Count_Per_County&State.csv')